In [5]:
### Chapter 7: compulsory: 3 and 5 and select one of these two sets: {1} or {2}
from __future__ import division
from __future__ import print_function
import pandas as pd
%pylab inline
import time
import warnings
import os
from collections import Counter
from itertools import chain
import json
import seaborn as sns
import ast
from itertools import izip

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
import sklearn.preprocessing

print("Imports done ...✈✈✈✈✈✈✈✈ You are free to go.")

import six


Populating the interactive namespace from numpy and matplotlib
Imports done ...✈✈✈✈✈✈✈✈ You are free to go.


In [6]:
def pairwise(iterable):
    "s -> (s0, s1), (s2, s3), (s4, s5), ..."
    a = iter(iterable)
    return izip(a, a)

def get_labels(fname):
    label_mappings = {}
    with open(fname,'r+') as f:
        for line in f.readlines():
            line = line.rstrip().lower()
            
            splitted = line.split('|')
            splitted = [x.strip().rstrip().lower() for x in splitted]
            if len(splitted) == 2:
                if splitted[1] =='label':
                    continue
                label = int(splitted[0])
                name = splitted[1].rstrip().lower()
                if name  not in label_mappings.keys():
                    label_mappings[name] = []
                    label_mappings[name].append(label)
                else:
                    label_mappings[name].append(label)
            else:
                break
    return label_mappings
                




def generate_labels(header=['sensor_type','device','label','label_start','datetime_start','label_end','datetime_end'],
                    fname='./ml4qsbeasts/log_events',log_fname='./ml4qsbeasts/new_log',fout='./ml4qsbeasts/labels.csv'):
    
    label_mapping = get_labels(log_fname)
    labels_list = []
    lines_list = []
    with open(fname,'r+') as f:
        for line in f.readlines():
            lines_list.append(line)
    
    for line,line2 in pairwise(lines_list):
 
        splitted = line.split('|')
        splitted = [x.rstrip().lower() for x in splitted]
        label,_,_,time_start = splitted

        splitted = line2.split('|')
        splitted = [x.rstrip().lower() for x in splitted]
        label,_,_,time_end = splitted
        for k in label_mapping.keys():
            if int(label) in label_mapping[k]:
                key = k
        
        labels_list.append(['interval_label','tommie smartphone',
                              key,time_start,pd.to_datetime(time_start,unit='ms'),
                            time_end,pd.to_datetime(time_end,unit='ms')])
        
    
    with open(fout,'w+') as f:
        f.write(','.join([str(x) for x in header])+'\n')
        for line in labels_list:
            line = ','.join([str(x) for x in line])
            f.write(line+'\n')
    return labels_list

            
def get_sensor_values(fname,device='log1'):
    '''Get sensor values function. Parses the .txt log and puts all sensors in a csv
        args:
            fname the filename of the sensor log
        
        output:
            sensor_dict a dictionary where the keys are the sensors and the values are label-vector-timestamp
    '''
    sensor_dict = {}
    device = fname.split('/')[len(fname.split('/'))-1]
    labels_dict = {}
    label_names = get_labels(fname)
    print(label_names)
    with open(fname,'r+') as f:
        for line in f.readlines():
            splitted = line.split('|')
            if len(splitted) != 4:
                #in case we are before our data to avoid parsing errors
                continue
            splitted = [x.rstrip().lower() for x in splitted]
            label_id,sensorname,vector,timestamp = splitted
            if sensorname == 'sensorname':
                continue
            if sensorname not in sensor_dict.keys():
                sensor_dict[sensorname] = []
            else:
                tmp_list = sensor_dict.get(sensorname)
                tmp_list.append([label_id,vector,timestamp])
                sensor_dict[sensorname] = tmp_list
            
            
    return sensor_dict

In [14]:

dataset_folder = './ml4qsbeasts/'

fname = 'new_log'
dataset_path = os.path.join(dataset_folder,fname)
csv_folder = os.path.join(dataset_folder,'csv')

In [15]:
sensor_dict = get_sensor_values(dataset_path,device='tommie_device')
if not os.path.exists(csv_folder):
    os.mkdir(csv_folder)
generate_csv(sensor_dict['bmi160 accelerometer'],header=['x','y','z','timestamp'],fname='./ml4qsbeasts/csv/accelerometer.csv')
generate_csv(sensor_dict['bmi160 gyroscope'],header=['x','y','z','timestamp'],fname='./ml4qsbeasts/csv/gyroscope.csv')
generate_csv(sensor_dict['rotation vector'],header=['x','y','z','theta','phi','timestamp'],fname='./ml4qsbeasts/csv/rotation.csv')

dd = generate_labels()

{'standing': [6, 11], 'walking': [9, 12], 'sitting': [10, 13], 'stairsdown': [4, 7], 'stairsup': [5, 8], 'ondesk': [14]}


# Chapter 7.3
In our example, we use the same methods, but on a classification task with the same labels as the crowdsignal dataset.

In [11]:
print(len(dd))

11
